# A Data-Driven Approach To Cryptocurrency Speculation

*How do Bitcoin markets behave? What are the causes of the sudden spikes and dips in cryptocurrency values?  Are the markets for different altcoins, such as Litecoin and Ripple, inseparably linked or largely independent?  **How can we predict what will happen next?***

Articles on cryptocurrencies, such as Bitcoin and Ethereum, are rife with speculation these days, with hundreds of self-proclaimed experts advocating for the trends that they expect to emerge.  What is lacking from many of these analyses is a strong data analysis foundation to backup the claims. 

The goal of this article is to provide an easy introduction to cryptocurrency analysis using Python.  We will walk through a simple Python script to retrieve, analyze, and visualize data on different cryptocurrencies.  In the process, we will uncover an interesting trend in how these volatile markets behave, and how they are evolving.

<img id="altcoin_prices_combined_0" src="https://cdn.patricktriest.com/blog/images/posts/crypto-markets/plot-images/altcoin_prices_combined.png" alt="Combined Altcoin Prices">

This is not a post explaining what cryptocurrencies are (if you want one, I would recommend <a href="https://medium.com/tradecraft-traction/blockchain-for-the-rest-of-us-c3fc5e42254f" target="_blank" rel="noopener">this great overview</a>), nor is it an opinion piece on which specific currencies will rise and which will fall.  Instead, all that we are concerned about in this tutorial is procuring the raw data and uncovering the stories hidden in the numbers.

# Project Setup

Once you've got a blank Jupyter notebook open, the first thing we'll do is import the required dependencies.

In [1]:
import os
import numpy as np
import pandas as pd
import pickle
import quandl
from datetime import datetime

We'll also import Plotly and enable the offline mode.

In [2]:
import plotly.offline as py
import plotly.graph_objs as go
import plotly.figure_factory as ff
py.init_notebook_mode(connected=True)

In [3]:
quandl.ApiConfig.api_key = os.environ['QUANDL_API_KEY']
data_dir = 'data/'

# Retrieve Bitcoin Pricing Data
Now that everything is set up, we're ready to start retrieving data for analysis.  First, we need to get Bitcoin pricing data using [Quandl's free Bitcoin API](https://blog.quandl.com/api-for-bitcoin-data).

## Define Quandl Helper Function
To assist with this data retrieval we'll define a function to download and cache datasets from Quandl.

In [4]:
def get_quandl_data(quandl_id):
    '''Download and cache Quandl dataseries'''
    cache_path = '{}{}.pkl'.format(data_dir, quandl_id.replace('/','-'))
    try:
        f = open(cache_path, 'rb')
        df = pickle.load(f)   
        print('Loaded {} from cache'.format(quandl_id))
    except (OSError, IOError) as e:
        print('Downloading {} from Quandl'.format(quandl_id))
        df = quandl.get(quandl_id, returns="pandas")
        df.to_pickle(cache_path)
        print('Cached {} at {}'.format(quandl_id, cache_path))
    return df

We're using `pickle` to serialize and save the downloaded data as a file, which will prevent our script from re-downloading the same data each time we run the script.  The function will return the data as a [Pandas]('http://pandas.pydata.org/') dataframe.  If you're not familiar with dataframes, you can think of them as super-powered Python spreadsheets.

## Pull Kraken Exchange Pricing Data
Let's first pull the historical Bitcoin exchange rate for the [Kraken](https://www.kraken.com/) Bitcoin exchange.

In [5]:
# Pull Kraken BTC price exchange data
btc_usd_price_kraken = get_quandl_data('BCHARTS/KRAKENUSD')

Loaded BCHARTS/KRAKENUSD from cache


We can inspect the first 5 rows of the dataframe using the `head()` method.

In [6]:
btc_usd_price_kraken.head()

,Open,High,Low,Close,Volume (BTC),Volume (Currency),Weighted Price
Date,,,,,,,
2014-01-07,874.67040,892.06753,810.00000,810.00000,15.622378,13151.472844,841.835522
2014-01-08,810.00000,899.84281,788.00000,824.98287,19.182756,16097.329584,839.156269
2014-01-09,825.56345,870.00000,807.42084,841.86934,8.158335,6784.249982,831.572913
2014-01-10,839.99000,857.34056,817.00000,857.33056,8.024510,6780.220188,844.938794
2014-01-11,858.20000,918.05471,857.16554,899.84105,18.748285,16698.566929,890.671709


Next, we'll generate a simple chart as a quick visual verification that the data looks correct.  

In [7]:
# Chart the BTC pricing data
btc_trace = go.Scatter(x=btc_usd_price_kraken.index, y=btc_usd_price_kraken['Weighted Price'])
py.iplot([btc_trace])

Here, we're using [Plotly](https://plot.ly/) for generating our visualizations.  This is a less traditional choice than some of the more established Python data visualization libraries such as [Matplotlib](https://matplotlib.org/), but I think Plotly is a great choice since it produces fully-interactive charts using [D3.js](https://d3js.org/).  These charts have attractive visual defaults, are easy to explore, and are very simple to embed in web pages.

> As a quick sanity check, you should compare the generated chart with publically available graphs on Bitcoin prices(such as those on [Coinbase](https://www.coinbase.com/dashboard)), to verify that the downloaded data is legit.

## Pull Pricing Data From More BTC Exchanges
You might have noticed a hitch in this dataset - there are a few notable down-spikes, particularly in late 2014 and early 2016.  These spikes are specific to the Kraken dataset, and we obviously don't want them to be reflected in our overall pricing analysis.  

The nature of Bitcoin exchanges is that the pricing is determined by supply and demand, hence no single exchange contains a true "master price" of Bitcoin.  To solve this issue, along with that of down-spikes, we'll pull data from three more major Bitcoin changes to calculate an aggregate Bitcoin price index.

First, we will download the data from each exchange into a dictionary of dataframes.

In [8]:
# Pull pricing data for 3 more BTC exchanges
exchanges = ['COINBASE','BITSTAMP','ITBIT']

exchange_data = {}

exchange_data['KRAKEN'] = btc_usd_price_kraken

for exchange in exchanges:
    exchange_code = 'BCHARTS/{}USD'.format(exchange)
    btc_exchange_df = get_quandl_data(exchange_code)
    exchange_data[exchange] = btc_exchange_df

Loaded BCHARTS/COINBASEUSD from cache
Loaded BCHARTS/BITSTAMPUSD from cache
Loaded BCHARTS/ITBITUSD from cache


## Merge All Of The Pricing Data Into A Single Dataframe
Next, we will define a simple function to merge a common column of each dataframe into a new combined dataframe.

In [9]:
def merge_dfs_on_column(dataframes, labels, col):
    '''Merge a single column of each dataframe into a new combined dataframe'''
    series_dict = {}
    for index in range(len(dataframes)):
        series_dict[labels[index]] = dataframes[index][col]
        
    return pd.DataFrame(series_dict)

Now we will merge all of the dataframes together on their "Weighted Price" column.

In [10]:
# Merge the BTC price dataseries' into a single dataframe
btc_usd_datasets = merge_dfs_on_column(list(exchange_data.values()), list(exchange_data.keys()), 'Weighted Price')

Finally, we can preview last five rows the result using the `tail()` method, to make sure it looks ok.

In [11]:
btc_usd_datasets.tail()

,BITSTAMP,COINBASE,ITBIT,KRAKEN
Date,,,,
2017-12-18,18634.418590,18779.790859,18640.346919,18683.675327
2017-12-19,17986.053806,18046.801749,17934.823780,18093.126238
2017-12-20,16724.840365,16627.512506,16644.736331,16515.544624
2017-12-21,16033.611116,16072.079918,15940.292149,15707.989038
2017-12-22,15671.434483,NaN,NaN,15205.831352


## Visualize The Pricing Datasets
The next logical step is to visualize how these pricing datasets compare.  For this, we'll define a helper function to provide a single-line command to compare each column in the dataframe on a graph using Plotly.

In [12]:
def df_scatter(df, title, seperate_y_axis=False, y_axis_label='', scale='linear', initial_hide=False):
    '''Generate a scatter plot of the entire dataframe'''
    label_arr = list(df)
    series_arr = list(map(lambda col: df[col], label_arr))
    
    layout = go.Layout(
        title=title,
        legend=dict(orientation="h"),
        xaxis=dict(type='date'),
        yaxis=dict(
            title=y_axis_label,
            showticklabels= not seperate_y_axis,
            type=scale
        )
    )
    
    y_axis_config = dict(
        overlaying='y',
        showticklabels=False,
        type=scale )
    
    visibility = 'visible'
    if initial_hide:
        visibility = 'legendonly'
        
    # Form Trace For Each Series
    trace_arr = []
    for index, series in enumerate(series_arr):
        trace = go.Scatter(
            x=series.index, 
            y=series, 
            name=label_arr[index],
            visible=visibility
        )
        
        # Add seperate axis for the series
        if seperate_y_axis:
            trace['yaxis'] = 'y{}'.format(index + 1)
            layout['yaxis{}'.format(index + 1)] = y_axis_config    
        trace_arr.append(trace)

    fig = go.Figure(data=trace_arr, layout=layout)
    py.iplot(fig)

In the interest of brevity, I won't go too far into how this helper function works.  Check out the documentation for [Pandas](http://pandas.pydata.org/) and [Plotly](https://plot.ly/) if you would like to learn more. 

With the function defined, we can compare our pricing datasets like so.

In [13]:
# Plot all of the BTC exchange prices
df_scatter(btc_usd_datasets, 'Bitcoin Price (USD) By Exchange')

## Clean and Aggregate the Pricing Data
We can see that, although the four series follow roughly the same path, there are various irregularities in each that we'll want to get rid of.

Let's remove all of the zero values from the dataframe, since we know that the price of Bitcoin has never been equal to zero in the timeframe that we are examining.

In [14]:
# Remove "0" values
btc_usd_datasets.replace(0, np.nan, inplace=True)

When we re-chart the dataframe, we'll see a much cleaner looking chart without the spikes.

In [15]:
# Plot the revised dataframe
df_scatter(btc_usd_datasets, 'Bitcoin Price (USD) By Exchange')

We can now calculate a new column, containing the daily average Bitcoin price across all of the exchanges.

In [16]:
# Calculate the average BTC price as a new column
btc_usd_datasets['avg_btc_price_usd'] = btc_usd_datasets.mean(axis=1)

This new column is our Bitcoin pricing index!  Let's chart that column to make sure it looks ok.

In [17]:
# Plot the average BTC price
btc_trace = go.Scatter(x=btc_usd_datasets.index, y=btc_usd_datasets['avg_btc_price_usd'])
py.iplot([btc_trace])

Yup, looks good.  We'll use this aggregate pricing series later on, in order to convert the exchange rates of other cryptocurrencies to USD.

# Retrieve Altcoin Pricing Data
Now that we have a solid time series dataset for the price of Bitcoin, let's pull in some data on non-Bitcoin cryptocurrencies, commonly referred to as altcoins.

## Define Poloniex API Helper Functions

For retrieving data on cryptocurrencies we'll be using the [Poloniex API](https://poloniex.com/support/api/).  To assist in the altcoin data retrieval, we'll define two helper functions to download and cache JSON data from this API.

First, we'll define `get_json_data`, which will download and cache JSON data from a provided URL.

In [18]:
def get_json_data(json_url, file_name):
    '''Download and cache JSON data, return as a dataframe.'''
    cache_path = '{}{}.pkl'.format(data_dir, file_name)
    try:        
        f = open(cache_path, 'rb')
        df = pickle.load(f)   
        print('Loaded {} from cache'.format(file_name))
    except (OSError, IOError) as e:
        print('Downloading {}'.format(json_url))
        df = pd.read_json(json_url)
        df.to_pickle(cache_path)
        print('Cached response at {}'.format(cache_path))
    return df

Next, we'll define a function to format Poloniex API HTTP requests and call our new `get_json_data` function to save the resulting data.

In [19]:
base_polo_url = 'https://poloniex.com/public?command=returnChartData&currencyPair={}&start={}&end={}&period={}'
start_date = datetime.strptime('2015-01-01', '%Y-%m-%d') # get data from the start of 2015
end_date = datetime.now() # up until today
pediod = 86400 # pull daily data (86,400 seconds per day)

def get_crypto_data(poloniex_pair):
    '''Retrieve cryptocurrency data from poloniex'''
    json_url = base_polo_url.format(poloniex_pair, start_date.timestamp(), end_date.timestamp(), pediod)
    data_df = get_json_data(json_url, poloniex_pair)
    data_df = data_df.set_index('date')
    return data_df

This function will take a cryptocurrency pair string (such as 'BTC_ETH') and return the dataframe containing the historical exchange rate of the two currencies.

## Download Trading Data From Poloniex
Most altcoins cannot be bought directly with USD; to acquire these coins individuals often buy Bitcoins and then trade the Bitcoins for altcoins on cryptocurrency exchanges.  For this reason we'll be downloading the exchange rate to BTC for each coin, and then we'll use our existing BTC pricing data to convert this value to USD.

We'll download exchange data for nine of the top cryptocurrencies -
[Ethereum](https://www.ethereum.org/), [Litecoin](https://litecoin.org/), [Ripple](https://ripple.com/), [Ethereum Classic](https://ethereumclassic.github.io/), [Stellar](https://www.stellar.org/), [Dashcoin](http://dashcoin.info/), [Siacoin](http://sia.tech/), [Monero](https://getmonero.org/), and [NEM](https://www.nem.io/).

In [20]:
altcoins = ['ETH','LTC','XRP','ETC','STR','DASH','SC','XMR','XEM']

altcoin_data = {}
for altcoin in altcoins:
    coinpair = 'BTC_{}'.format(altcoin)
    crypto_price_df = get_crypto_data(coinpair)
    altcoin_data[altcoin] = crypto_price_df

Cached response at data/BTC_ETH.pkl
Cached response at data/BTC_LTC.pkl
Cached response at data/BTC_XRP.pkl
Cached response at data/BTC_ETC.pkl
Cached response at data/BTC_STR.pkl
Cached response at data/BTC_DASH.pkl
Cached response at data/BTC_SC.pkl
Cached response at data/BTC_XMR.pkl
Cached response at data/BTC_XEM.pkl


Now we have a dictionary of 9 dataframes, each containing the historical daily average exchange prices between the altcoin and Bitcoin.

We can preview the last few rows of the Ethereum price table to make sure it looks ok.

In [21]:
altcoin_data['ETH'].tail()

,close,high,low,open,quoteVolume,volume,weightedAverage
date,,,,,,,
2017-12-26,0.047869,0.053496,0.046001,0.052154,111268.712134,5455.818989,0.049033
2017-12-27,0.047820,0.048950,0.046195,0.047869,70886.653227,3386.616949,0.047775
2017-12-28,0.049760,0.050620,0.047717,0.047883,80968.926518,3996.575786,0.049359
2017-12-29,0.051220,0.051700,0.049008,0.049760,87373.758406,4425.826509,0.050654
2017-12-30,0.051050,0.051611,0.050889,0.051161,10982.711712,562.581236,0.051224


## Convert Prices to USD

Since we now have the exchange rate for each cryptocurrency to Bitcoin, and we have the Bitcoin/USD historical pricing index, we can directly calculate the USD price series for each altcoin.

In [22]:
# Calculate USD Price as a new column in each altcoin dataframe
for altcoin in altcoin_data.keys():
    altcoin_data[altcoin]['price_usd'] =  altcoin_data[altcoin]['weightedAverage'] * btc_usd_datasets['avg_btc_price_usd']

Here, we've created a new column in each altcoin dataframe with the USD prices for that coin.

Next, we can re-use our `merge_dfs_on_column` function from earlier to create a combined dataframe of the USD price for each cryptocurrency.

In [23]:
# Merge USD price of each altcoin into single dataframe 
combined_df = merge_dfs_on_column(list(altcoin_data.values()), list(altcoin_data.keys()), 'price_usd')

Easy.  Now let's also add the Bitcoin prices as a final column to the combined dataframe.

In [24]:
# Add BTC price to the dataframe
combined_df['BTC'] = btc_usd_datasets['avg_btc_price_usd']

Now we should have a single dataframe containing daily USD prices for the ten cryptocurrencies that we're examining.

Let's reuse our `df_scatter` function from earlier to chart all of the cryptocurrency prices against each other.

In [25]:
# Chart all of the altocoin prices
df_scatter(combined_df, 'Cryptocurrency Prices (USD)', seperate_y_axis=False, y_axis_label='Coin Value (USD)', scale='log')

Nice! This graph gives a pretty solid "big picture" view of how the exchange rates of each currency have varied over the past few years.  

> Note that we're using a logarithmic y-axis scale in order to compare all of currencies on the same plot.  You are welcome to try out different parameters values here (such as `scale='linear'`) to get different perspectives on the data.

# Compute Correlation Values of The Cryptocurrencies
You might notice is that the cryptocurrency exchange rates, despite their wildly different values and volatility, seem to be slightly correlated. Especially since the spike in April 2017, even many of the smaller fluctuations appear to be occurring in sync across the entire market.  

A visually-derived hunch is not much better than a guess until we have the stats to back it up.

We can test our correlation hypothesis using the Pandas `corr()` method, which computes a Pearson correlation coefficient for each column in the dataframe against each other column.

Computing correlations directly on a non-stationary time series (such as raw pricing data) can give biased correlation values.  We will work around this by using the `pct_change()` method, which will convert each cell in the dataframe from an absolute price value to a daily return percentage.

## Correlations in 2016
First we'll calculate correlations for 2016.

In [26]:
# Calculate the pearson correlation coefficients for altcoins in 2016
combined_df_2016 = combined_df[combined_df.index.year == 2016]
combined_df_2016.pct_change().corr(method='pearson')

,DASH,ETC,ETH,LTC,SC,STR,XEM,XMR,XRP,BTC
DASH,1.000000,0.003992,0.122695,-0.012194,0.026602,0.058083,0.014571,0.121537,0.088657,-0.014040
ETC,0.003992,1.000000,-0.181991,-0.131079,-0.008066,-0.102654,-0.080938,-0.105898,-0.054095,-0.170538
ETH,0.122695,-0.181991,1.000000,-0.064652,0.169642,0.035093,0.043205,0.087216,0.085630,-0.006502
LTC,-0.012194,-0.131079,-0.064652,1.000000,0.012253,0.113523,0.160667,0.129475,0.053712,0.750174
SC,0.026602,-0.008066,0.169642,0.012253,1.000000,0.143252,0.106153,0.047910,0.021098,0.035116
STR,0.058083,-0.102654,0.035093,0.113523,0.143252,1.000000,0.225132,0.027998,0.320116,0.079075
XEM,0.014571,-0.080938,0.043205,0.160667,0.106153,0.225132,1.000000,0.016438,0.101326,0.227674
XMR,0.121537,-0.105898,0.087216,0.129475,0.047910,0.027998,0.016438,1.000000,0.027649,0.127520
XRP,0.088657,-0.054095,0.085630,0.053712,0.021098,0.320116,0.101326,0.027649,1.000000,0.044161
BTC,-0.014040,-0.170538,-0.006502,0.750174,0.035116,0.079075,0.227674,0.127520,0.044161,1.000000


These correlation coefficients are all over the place.  Coefficients close to 1 or -1 mean that the series' are strongly correlated or inversely correlated respectively, and coefficients close to zero mean that the values tend to fluctuate independently of each other.

## Heatmap Visualization
To help visualize these results, we'll create one more helper visualization function.

In [27]:
def correlation_heatmap(df, title, absolute_bounds=True):
    '''Plot a correlation heatmap for the entire dataframe'''
    heatmap = go.Heatmap(
        z=df.corr(method='pearson').as_matrix(),
        x=df.columns,
        y=df.columns,
        colorbar=dict(title='Pearson Coefficient'),
    )
    
    layout = go.Layout(title=title)
    
    if absolute_bounds:
        heatmap['zmax'] = 1.0
        heatmap['zmin'] = -1.0
        
    fig = go.Figure(data=[heatmap], layout=layout)
    py.iplot(fig)

In [28]:
correlation_heatmap(combined_df_2016.pct_change(), "Cryptocurrency Correlations in 2016")

Here, the dark red values represent strong correlations (note that each currency is, obviously, strongly correlated with itself), and the dark blue values represent strong inverse correlations.  All of the light blue/orange/gray/tan colors in-between represent varying degrees of weak/non-existent correlations.

What does this chart tell us? Essentially, it shows that there was very little statistically significant linkage between how the prices of different cryptocurrencies fluctuated during 2016.

## Correlations in 2017
Now, to test our hypothesis that the cryptocurrencies have become more correlated in recent months, let's repeat the same test using only the data from 2017.

In [29]:
combined_df_2017 = combined_df[combined_df.index.year == 2017]
combined_df_2017.pct_change().corr(method='pearson')

,DASH,ETC,ETH,LTC,SC,STR,XEM,XMR,XRP,BTC
DASH,1.000000,0.356029,0.483985,0.307250,0.261717,0.172162,0.301643,0.468115,0.088482,0.253301
ETC,0.356029,1.000000,0.582418,0.459576,0.282777,0.200511,0.301397,0.419364,0.112664,0.374537
ETH,0.483985,0.582418,1.000000,0.415845,0.355329,0.249987,0.379089,0.533111,0.215695,0.375294
LTC,0.307250,0.459576,0.415845,1.000000,0.315951,0.306738,0.360324,0.408562,0.341336,0.386835
SC,0.261717,0.282777,0.355329,0.315951,1.000000,0.404482,0.296425,0.350172,0.242257,0.297601
STR,0.172162,0.200511,0.249987,0.306738,0.404482,1.000000,0.330350,0.323703,0.495737,0.227773
XEM,0.301643,0.301397,0.379089,0.360324,0.296425,0.330350,1.000000,0.312540,0.267936,0.306711
XMR,0.468115,0.419364,0.533111,0.408562,0.350172,0.323703,0.312540,1.000000,0.226901,0.356728
XRP,0.088482,0.112664,0.215695,0.341336,0.242257,0.495737,0.267936,0.226901,1.000000,0.142041
BTC,0.253301,0.374537,0.375294,0.386835,0.297601,0.227773,0.306711,0.356728,0.142041,1.000000


These are somewhat more significant correlation coefficients.  Strong enough to use as the sole basis for an investment? Certainly not.  

It is notable, however, that almost all of the cryptocurrencies have become more correlated with each other across the board.

In [30]:
correlation_heatmap(combined_df_2017.pct_change(), "Cryptocurrency Correlations in 2017")

Huh. That's rather interesting.

## Why is this happening?

Good question.  I'm really not sure.  

The most immediate explanation that comes to mind is that **hedge funds have recently begun publicly trading in crypto-currency markets**[^1][^2].  These funds have vastly more capital to play with than the average trader, so if a fund is hedging their bets across multiple cryptocurrencies, and using similar trading strategies for each based on independent variables (say, the stock market), it could make sense that this trend would emerge.


## In-Depth - XRP and STR
For instance, one noticeable trait of the above chart is that XRP (the token for [Ripple](https://ripple.com/)), is the least correlated cryptocurrency.  The notable exception here is with STR (the token for [Stellar](https://www.stellar.org/), officially known as "Lumens"), which has a stronger (0.62) correlation with XRP.  

What is interesting here is that Stellar and Ripple are both fairly similar fintech platforms aimed at reducing the friction of international money transfers between banks.  

It is conceivable that some big-money players and hedge funds might be using similar trading strategies for their investments in Stellar and Ripple, due to the similarity of the blockchain services that use each token. This could explain why XRP is so much more heavily correlated with STR than with the other cryptocurrencies.

> Quick Plug - I'm a contributor to [Chipper](https://www.chipper.xyz/), a (very) early-stage startup using Stellar with the aim of disrupting micro-remittances in Africa.


# Your Turn

This explanation is, however, largely speculative.  **Maybe you can do better**.  With the foundation we've made here, there are hundreds of different paths to take to continue searching for stories within the data.  

Here are some ideas:

- Add data from more cryptocurrencies to the analysis.
- Adjust the time frame and granularity of the correlation analysis, for a more fine or coarse grained view of the trends. 
- Search for trends in trading volume and/or blockchain mining data sets.  The buy/sell volume ratios are likely more relevant than the raw price data if you want to predict future price fluctuations.
- Add pricing data on stocks, commodities, and fiat currencies to determine which of them correlate with cryptocurrencies (but please remember the old adage that "Correlation does not imply causation").
- Quantify the amount of "buzz" surrounding specific cryptocurrencies using Event Registry, GDLELT, and Google Trends. 
- Train a predictive machine learning model on the data to predict tomorrow's prices.  If you're more ambitious, you could even try doing this with a recurrent neural network (RNN).
- Use your analysis to create an automated "Trading Bot" on a trading site such as Poloniex or Coinbase, using their respective trading APIs.  Be careful: a poorly optimized trading bot is an easy way to lose your money quickly.
- **Share your findings!**  The best part of Bitcoin, and of cryptocurrencies in general, is that their decentralized nature makes them more free and democratic than virtually any other market.  Open source your analysis, participate in the community, maybe write a blog post about it.

Hopefully, now you have the skills to do your own analysis and to think critically about any speculative cryptocurrency articles you might read in the near future, especially those written without any data to back up the provided predictions.

Thanks for reading, and feel free to comment below with any ideas, suggestions, or criticisms regarding this tutorial.  I've got second (and potentially third) part in the works, which will likely be following through on some of same the ideas listed above, so stay tuned for more in the coming weeks.

[^1]: http://fortune.com/2017/07/26/bitcoin-cryptocurrency-hedge-fund-sequoia-andreessen-horowitz-metastable/
[^2]: https://www.forbes.com/sites/laurashin/2017/07/12/crypto-boom-15-new-hedge-funds-want-in-on-84000-returns/#7946ab0d416a